In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
import pmdarima as pm
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import os
import logging
from datetime import datetime
import warnings
from tqdm import tqdm
import seaborn as sns
from joblib import Parallel, delayed
import time

img_dir = 'classical_model_results'
# Tạo thư mục lưu biểu đồ nếu chưa tồn tại
os.makedirs(img_dir, exist_ok=True)

# Thiết lập logging
logging.basicConfig(
    filename=f'{img_dir}/classical_models_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")

# Tham số cấu hình
CONFIG = {
    'forecast_horizon': 12,
    'seasonal_periods': 12,
    'min_data_length': 24,
    'img_dir': img_dir,
    'results_file': f'{img_dir}/classical_model_results.csv',
    'n_jobs': -1,  # Số lượng CPU cores để chạy song song (-1: tất cả cores)
}


def validate_input_data(df, required_columns):
    """Kiểm tra tính hợp lệ của dữ liệu đầu vào."""
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Thiếu các cột bắt buộc: {required_columns}")
    if df.index.duplicated().any():
        raise ValueError("Index chứa giá trị trùng lặp!")
    if not df.index.is_monotonic_increasing:
        raise ValueError("Index không được sắp xếp tăng dần!")
    if df[required_columns].isnull().sum().any():
        raise ValueError(f"Dữ liệu chứa giá trị thiếu: {df[required_columns].isnull().sum().to_dict()}")
    if df[required_columns].replace([np.inf, -np.inf], np.nan).isnull().sum().any():
        raise ValueError("Dữ liệu chứa giá trị vô cực!")
    if not all(df[required_columns].dtypes.apply(lambda x: np.issubdtype(x, np.number))):
        raise ValueError("Một số cột không phải kiểu số!")

def check_stationarity(series, name):
    """Kiểm tra tính dừng bằng ADF test và trả về bậc sai phân cần thiết."""
    max_diff = 2
    series_clean = series.dropna().replace([np.inf, -np.inf], np.nan).dropna()
    if len(series_clean) < 2:
        logger.error(f"{name}: Dữ liệu quá ngắn sau khi làm sạch!")
        return 0
    d = 0
    while d <= max_diff:
        result = adfuller(series_clean)
        logger.info(f"ADF Test for {name} (d={d}): Statistic={result[0]:.4f}, p-value={result[1]:.4f}")
        if result[1] < 0.05:
            logger.info(f"{name} dừng ở bậc sai phân d={d}")
            return d
        if d == max_diff:
            logger.warning(f"{name} vẫn không dừng sau {max_diff} lần sai phân. Dùng d={d}.")
            return d
        series_clean = series_clean.diff().dropna()
        if len(series_clean) < 2:
            logger.warning(f"{name}: Dữ liệu quá ngắn sau sai phân {d+1}!")
            return d
        d += 1
    return d

def calculate_metrics(actual, predicted):
    """Tính RMSE, MAE, MAPE, sMAPE, NormMAPE, và Directional Accuracy."""
    actual = np.array(actual, dtype=float)
    predicted = np.array(predicted, dtype=float)
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted) & ~np.isinf(actual) & ~np.isinf(predicted)
    actual = actual[valid_mask]
    predicted = predicted[valid_mask]
    
    if len(actual) == 0:
        logger.warning("Không có dữ liệu hợp lệ để tính chỉ số!")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = mean_absolute_percentage_error(actual, predicted) * 100 if np.all(np.abs(actual) > 1e-8) else np.nan
    smape = 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))
    norm_mape = mape / np.mean(np.abs(actual)) if not np.isnan(mape) else np.nan
    
    # Tính Directional Accuracy
    actual_diff = np.diff(actual)
    pred_diff = np.diff(predicted)
    directional_acc = np.mean((actual_diff * pred_diff) > 0) * 100 if len(actual_diff) > 0 else np.nan
    
    return rmse, mae, mape, smape, norm_mape, directional_acc

def plot_forecast(historical, test, forecast, forecast_index, title, ylabel, filename, confidence_intervals=None):
    """Vẽ và lưu biểu đồ dự báo với khoảng tin cậy."""
    plt.figure(figsize=(12, 6))
    plt.plot(historical.index, historical, label='Lịch sử', color='blue')
    plt.plot(test.index, test, label='Thực tế (Test)', color='green')
    plt.plot(forecast_index, forecast, label='Dự báo', color='orange', linestyle='--', linewidth=2)
    if confidence_intervals:
        plt.fill_between(
            forecast_index,
            confidence_intervals[0],
            confidence_intervals[1],
            color='orange',
            alpha=0.2,
            label='Khoảng tin cậy 95%'
        )
    plt.title(title)
    plt.xlabel('Thời gian')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename))
        logger.info(f"Đã lưu biểu đồ: {filename}")
    except Exception as e:
        logger.error(f"Lỗi khi lưu biểu đồ {filename}: {str(e)}")
    plt.close()

def plot_comparison_forecasts(historical, test, forecasts, forecast_index, title, ylabel, filename, metrics=None):
    """Vẽ so sánh dự báo từ nhiều mô hình với chú thích chỉ số RMSE."""
    plt.figure(figsize=(14, 8))
    plt.plot(historical.index, historical, label='Lịch sử', color='blue')
    plt.plot(test.index, test, label='Thực tế (Test)', color='green')
    colors = sns.color_palette("husl", len(forecasts))
    for (model_name, forecast), color in zip(forecasts.items(), colors):
        rmse = metrics.get(model_name, {}).get('RMSE', np.nan) if metrics else np.nan
        if forecast is None or pd.isna(rmse):
            logger.warning(f"Bỏ qua {model_name} trong biểu đồ so sánh do thiếu dự báo hoặc RMSE")
            continue
        label = f'Dự báo {model_name} (RMSE: {rmse:.4f})'
        plt.plot(forecast_index, forecast, label=label, linestyle='--', color=color)
    plt.title(title)
    plt.xlabel('Thời gian')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename))
        logger.info(f"Đã lưu biểu đồ so sánh: {filename}")
    except Exception as e:
        logger.error(f"Lỗi khi lưu biểu đồ so sánh {filename}: {str(e)}")
    plt.close()

def plot_residual_acf(residuals, title, filename):
    """Vẽ và lưu biểu đồ ACF của phần dư."""
    if residuals is None or len(residuals) < 2:
        logger.warning(f"Bỏ qua ACF: Không đủ dữ liệu phần dư - {title}")
        return
    plt.figure(figsize=(5, 3))
    max_lags = min(20, len(residuals) - 1)
    if max_lags < 1:
        logger.warning(f"Bỏ qua ACF: Quá ít residuals - {title}")
        return
    try:
        plot_acf(residuals, lags=max_lags, title=title)
        plt.tight_layout()
        plt.savefig(os.path.join(CONFIG['img_dir'], filename))
        logger.info(f"Đã lưu biểu đồ ACF: {filename}")
    except Exception as e:
        logger.error(f"Lỗi khi lưu biểu đồ ACF {filename}: {str(e)}")
    finally:
        plt.close()

def run_exponential_smoothing(train, test, forecast_index, seasonal_periods=CONFIG['seasonal_periods']):
    """Chạy mô hình Exponential Smoothing."""
    start_time = time.time()
    try:
        model = ExponentialSmoothing(
            train, trend='add', seasonal='add', seasonal_periods=seasonal_periods
        ).fit(optimized=True)
        forecast = model.forecast(CONFIG['forecast_horizon'])
        residuals = train - model.fittedvalues
        forecast = pd.Series(forecast.values, index=forecast_index)
        resid_std = np.std(residuals)
        ci_lower = forecast - 1.96 * resid_std
        ci_upper = forecast + 1.96 * resid_std
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        elapsed_time = time.time() - start_time
        logger.info(f"Exponential Smoothing: RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}, sMAPE={smape:.4f}, NormMAPE={norm_mape:.4f}, DirAcc={dir_acc:.4f}, Time={elapsed_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, (ci_lower, ci_upper)
    except Exception as e:
        logger.error(f"Lỗi Exponential Smoothing: {str(e)}")
        return None, None, None, None, None, None, None, None, None

def run_arima(train, test, forecast_index):
    """Chạy mô hình ARIMA với auto_arima để tìm tham số tối ưu."""
    start_time = time.time()
    try:
        model = pm.auto_arima(
            train, start_p=0, start_q=0, max_p=3, max_q=3, max_d=2,
            seasonal=False, trace=False, error_action='ignore', suppress_warnings=True,
            stepwise=True, information_criterion='aic'
        )
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - model.predict_in_sample()
        forecast = pd.Series(forecast, index=forecast_index)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        elapsed_time = time.time() - start_time
        logger.info(f"ARIMA (order={model.order}): RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}, sMAPE={smape:.4f}, NormMAPE={norm_mape:.4f}, DirAcc={dir_acc:.4f}, Time={elapsed_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Lỗi ARIMA: {str(e)}")
        return None, None, None, None, None, None, None, None, None

def run_sarima(train, test, forecast_index):
    """Chạy mô hình SARIMA với auto_arima để tìm tham số tối ưu."""
    start_time = time.time()
    try:
        model = pm.auto_arima(
            train, start_p=0, start_q=0, max_p=3, max_q=3, max_d=2,
            seasonal=True, m=CONFIG['seasonal_periods'],
            start_P=0, start_Q=0, max_P=2, max_Q=2, max_D=1,
            trace=False, error_action='ignore', suppress_warnings=True,
            stepwise=True, information_criterion='aic'
        )
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - model.predict_in_sample()
        forecast = pd.Series(forecast, index=forecast_index)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        elapsed_time = time.time() - start_time
        logger.info(f"SARIMA (order={model.order}, seasonal_order={model.seasonal_order}): RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}, sMAPE={smape:.4f}, NormMAPE={norm_mape:.4f}, DirAcc={dir_acc:.4f}, Time={elapsed_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Lỗi SARIMA: {str(e)}")
        return None, None, None, None, None, None, None, None, None

def run_sarimax(train, test, forecast_index, exog_train, exog_test):
    """Chạy mô hình SARIMAX với auto_arima để tìm tham số tối ưu và biến ngoại sinh."""
    start_time = time.time()
    try:
        # Kiểm tra dữ liệu ngoại sinh
        if exog_train.isna().any().any() or exog_test.isna().any().any():
            raise ValueError("Dữ liệu ngoại sinh chứa giá trị thiếu!")
        if len(exog_train) != len(train) or len(exog_test) != CONFIG['forecast_horizon']:
            raise ValueError("Kích thước dữ liệu ngoại sinh không khớp!")
        
        model = pm.auto_arima(
            train, exogenous=exog_train,
            start_p=0, start_q=0, max_p=3, max_q=3, max_d=2,
            seasonal=True, m=CONFIG['seasonal_periods'],
            start_P=0, start_Q=0, max_P=2, max_Q=2, max_D=1,
            trace=False, error_action='ignore', suppress_warnings=True,
            stepwise=True, information_criterion='aic', maxiter=50
        )
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'], exogenous=exog_test)
        residuals = train - model.predict_in_sample(exogenous=exog_train)
        forecast = pd.Series(forecast, index=forecast_index)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        elapsed_time = time.time() - start_time
        logger.info(f"SARIMAX (order={model.order}, seasonal_order={model.seasonal_order}): RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}, sMAPE={smape:.4f}, NormMAPE={norm_mape:.4f}, DirAcc={dir_acc:.4f}, Time={elapsed_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Lỗi SARIMAX: {str(e)}")
        return None, None, None, None, None, None, None, None, None

def run_prophet(train, test, forecast_index):
    """Chạy mô hình Prophet."""
    start_time = time.time()
    try:
        df_train = pd.DataFrame({'ds': train.index, 'y': train.values})
        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False).fit(df_train)
        future = pd.DataFrame({'ds': forecast_index})
        forecast = model.predict(future)
        forecast_series = pd.Series(forecast['yhat'].values, index=forecast_index)
        residuals = train - model.predict(df_train)['yhat']
        ci_lower = pd.Series(forecast['yhat_lower'].values, index=forecast_index)
        ci_upper = pd.Series(forecast['yhat_upper'].values, index=forecast_index)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast_series)
        elapsed_time = time.time() - start_time
        logger.info(f"Prophet: RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}, sMAPE={smape:.4f}, NormMAPE={norm_mape:.4f}, DirAcc={dir_acc:.4f}, Time={elapsed_time:.2f}s")
        return forecast_series, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, (ci_lower, ci_upper)
    except Exception as e:
        logger.error(f"Lỗi Prophet: {str(e)}")
        return None, None, None, None, None, None, None, None, None

def run_var(train, test, forecast_index, maxlags=12):
    """Chạy mô hình VAR."""
    start_time = time.time()
    try:
        model = VAR(train).fit(maxlags=maxlags, ic='aic')
        forecast = model.forecast(train.values[-model.k_ar:], steps=CONFIG['forecast_horizon'])
        forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=train.columns)
        residuals = train - model.fittedvalues
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast_df[train.columns[0]])
        elapsed_time = time.time() - start_time
        logger.info(f"VAR: RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}, sMAPE={smape:.4f}, NormMAPE={norm_mape:.4f}, DirAcc={dir_acc:.4f}, Time={elapsed_time:.2f}s")
        return forecast_df[train.columns[0]], residuals[train.columns[0]], rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Lỗi VAR: {str(e)}")
        return None, None, None, None, None, None, None, None, None

def run_model_for_target(target, train, test, forecast_index, model_name, model_func, params):
    """Chạy một mô hình cho một mục tiêu cụ thể."""
    logger.info(f"Chạy {model_name} cho {target}")
    start_time = time.time()
    try:
        if model_name == 'VAR':
            forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                train[['cpi_mom', 'cpi_yoy']], test[[target]], forecast_index, **params
            )
        elif model_name == 'SARIMAX':
            exog_var = 'cpi_yoy' if target == 'cpi_mom' else 'cpi_mom'
            exog_train = train[[exog_var]]
            exog_test = test[[exog_var]].reindex(forecast_index)
            if exog_test.isna().any().any():
                logger.error(f"Dữ liệu ngoại sinh {exog_var} chứa giá trị thiếu trong giai đoạn dự báo!")
                return None
            forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                train[target], test[target], forecast_index, exog_train, exog_test, **params
            )
        else:
            forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                train[target], test[target], forecast_index, **params
            )
        
        if forecast is None or rmse is None:
            logger.error(f"{model_name} cho {target} không tạo được dự báo hoặc RMSE hợp lệ!")
            return None
        
        # Vẽ biểu đồ dự báo
        plot_forecast(
            train[target][-36:], test[target], forecast, forecast_index,
            f'{model_name} Forecast for {target}', target,
            f'{target}_{model_name}_forecast.png', ci
        )
        # Vẽ ACF phần dư
        if residuals is not None:
            plot_residual_acf(
                residuals.dropna(),
                f'ACF of Residuals - {model_name} ({target})',
                f'{target}_{model_name}_acf.png'
            )
        elapsed_time = time.time() - start_time
        logger.info(f"Hoàn thành {model_name} cho {target} trong {elapsed_time:.2f}s")
        return {
            'Target': target,
            'Model': model_name,
            'RMSE': rmse,
            'MAE': mae,
            'MAPE': mape,
            'sMAPE': smape,
            'NormMAPE': norm_mape,
            'DirAcc': dir_acc,
            'Forecast': forecast,
            'Residuals': residuals,
            'CI': ci
        }
    except Exception as e:
        logger.error(f"Lỗi khi chạy {model_name} cho {target}: {str(e)}")
        return None

def main():
    """Hàm chính để chạy các mô hình và lưu kết quả."""
    try:
        # Đọc dữ liệu
        data = pd.read_csv('data/analyzed_time_series.csv')
        data['time'] = pd.to_datetime(data['time'])
        data.set_index('time', inplace=True)
        required_columns = ['cpi_mom', 'cpi_yoy']
        validate_input_data(data, required_columns)
        
        # Chia dữ liệu
        train_size = len(data) - CONFIG['forecast_horizon']
        train, test = data[:train_size], data[train_size:]
        forecast_index = pd.date_range(start=test.index[0], periods=CONFIG['forecast_horizon'], freq='MS')
        
        # Danh sách mô hình
        models = {
            'ARIMA': (run_arima, {}),
            'Exponential Smoothing': (run_exponential_smoothing, {}),
            'Prophet': (run_prophet, {}),
            'SARIMA': (run_sarima, {}),
            'SARIMAX': (run_sarimax, {}),
            'VAR': (run_var, {}),
        }
        
        results = []
        forecasts_mom = {}
        forecasts_yoy = {}
        metrics_mom = {}
        metrics_yoy = {}
        
        # Chạy các mô hình song song cho mỗi target
        for target in ['cpi_mom', 'cpi_yoy']:
            logger.info(f"Chạy các mô hình cho {target}")
            tasks = [
                delayed(run_model_for_target)(
                    target, train, test, forecast_index, model_name, model_func, params
                )
                for model_name, (model_func, params) in models.items()
            ]
            model_results = Parallel(n_jobs=CONFIG['n_jobs'], verbose=1)(tasks)
            
            for result in model_results:
                if result is not None:
                    results.append({
                        'Target': result['Target'],
                        'Model': result['Model'],
                        'RMSE': result['RMSE'],
                        'MAE': result['MAE'],
                        'MAPE': result['MAPE'],
                        'sMAPE': result['sMAPE'],
                        'NormMAPE': result['NormMAPE'],
                        'DirAcc': result['DirAcc']
                    })
                    if result['Target'] == 'cpi_mom':
                        forecasts_mom[result['Model']] = result['Forecast']
                        metrics_mom[result['Model']] = {'RMSE': result['RMSE']}
                    else:
                        forecasts_yoy[result['Model']] = result['Forecast']
                        metrics_yoy[result['Model']] = {'RMSE': result['RMSE']}
                else:
                    logger.warning(f"Kết quả cho một mô hình của {target} là None, bỏ qua!")
        
        # Vẽ so sánh các mô hình
        for target, forecasts, metrics in [('cpi_mom', forecasts_mom, metrics_mom), ('cpi_yoy', forecasts_yoy, metrics_yoy)]:
            if not forecasts:
                logger.warning(f"Không có dự báo hợp lệ cho {target}, bỏ qua biểu đồ so sánh")
                continue
            plot_comparison_forecasts(
                train[target][-36:], test[target], forecasts, forecast_index,
                f'Comparison of Forecasts for {target}', target,
                f'{target}_model_comparison.png', metrics=metrics
            )
        
        # Lưu kết quả
        results_df = pd.DataFrame(results)
        print(results_df)
        results_df.to_csv(CONFIG['results_file'], index=False)
        logger.info(f"Kết quả đã được lưu vào {CONFIG['results_file']}")
        
        # Lưu dự báo kết hợp
        for target, forecasts in [('cpi_mom', forecasts_mom), ('cpi_yoy', forecasts_yoy)]:
            if not forecasts:
                logger.warning(f"Không có dự báo hợp lệ cho {target}, bỏ qua lưu dự báo kết hợp")
                continue
            combined_forecast = pd.DataFrame({'Date': forecast_index})
            for model_name, forecast in forecasts.items():
                combined_forecast[f'{model_name}_{target}'] = forecast
            combined_forecast.to_csv(f'{img_dir}/combined_forecast_{target}.csv', index=False)
            logger.info(f"Dự báo kết hợp cho {target} đã được lưu vào plots/combined_forecast_{target}.csv")
        
    except Exception as e:
        logger.error(f"Lỗi chương trình chính: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 3 out of 6 | elapsed:    4.9s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done 6 out of 6 | elapsed:   24.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 3 out of 6 | elapsed:    3.7s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done 6 out of 6 | elapsed:   29.3s finished


    Target                  Model      RMSE       MAE      MAPE     sMAPE  \
0  cpi_mom                  ARIMA  0.365397  0.302643  0.301934  0.301654   
1  cpi_mom  Exponential Smoothing  0.246269  0.204052  0.203666  0.203571   
2  cpi_mom                Prophet  0.271708  0.220355  0.219714  0.219652   
3  cpi_mom                 SARIMA  0.359880  0.294441  0.293687  0.293412   
4  cpi_mom                SARIMAX  0.359880  0.294441  0.293687  0.293412   
5  cpi_yoy                  ARIMA  0.532810  0.466997  0.450435  0.449429   
6  cpi_yoy  Exponential Smoothing  1.199787  0.887507  0.860838  0.854469   
7  cpi_yoy                Prophet  1.156847  0.927644  0.891325  0.897388   
8  cpi_yoy                 SARIMA  0.211096  0.171916  0.165465  0.165663   
9  cpi_yoy                SARIMAX  0.211096  0.171916  0.165465  0.165663   

   NormMAPE     DirAcc  
0  0.003012  63.636364  
1  0.002032  36.363636  
2  0.002192  36.363636  
3  0.002930  27.272727  
4  0.002930  27.272727  
5 